<a href="https://colab.research.google.com/github/kaorimiyazonoo/file_processing/blob/copying_files/Scientific_Visualizations_with_Python_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scientific Visualizations with Python

---

Welcome to the Scientific Visualizations with Python course with the Lane Medical Library!

**Audience**: Scientists or medical professionals interested in visualizing data using Python. Some introductory experience in Python is expected.

## Loading Some Data

Here, we are creating a `DataFrame` called `cal_housing` that will contain sample data provided in all Google Colab notebooks on housing prices in California. Each row represents one house. All of the variables are continuous (e.g., `latitude` or `median_income`).

We will create two discrete variables. The first is called `median_age`, and it will put the house into one of six bins based on the age of the house. The second is `id`, which is a unique value for each house.

The `.cut()` function in the `pandas` library is quite useful. It converts a continuous variable into a categorical variable by creating bins.

The `.index` property of a `DataFrame` contains unique identifiers for each row. Here, we add it as the `id` column to the data frame.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read data using the pandas read_csv function
cal_housing = pd.read_csv('sample_data/california_housing_train.csv')

# Create the median_age variable using the pandas cut function
cal_housing['median_age'] = pd.cut(
    cal_housing.housing_median_age, bins=[0, 10, 20, 30, 40, 50, 60],
    labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50-60'],
    include_lowest=True
)

# Store the index of the DataFrame as a variable, which assigns a unique value
# to each house
cal_housing['id'] = cal_housing.index

# Visualize the first few lines
cal_housing.head()

## Scatter Plot

A scatter plot is used to visualize the relationship between two continuous variables.

In [ ]:
import matplotlib.pyplot as plt

The `plt.scatter()` function generates a scatter plot. Recall from last session that the `median_house_value` is capped at 500,000. To account for this artifact, we will filter to records with a `median_house_value` of less than 500,000.

In this plot, we visualize the relationship between `median_income` and `median_house_value`.

In [ ]:
plot_data = cal_housing[cal_housing.median_house_value < 500_000]

plt.figure(figsize=(8, 8))

plt.scatter(plot_data.median_income, plot_data.median_house_value)

❓ **Try it yourself!** The relationship is challenging to visualize because there are so many points. Overplotting can make it hard to interpret the density of points in various regions. The `plt.scatter()` function has a `s` argument that controls the size of the points. Try setting this to `1`.

In [ ]:
# YOUR CODE HERE

❓ **Try it yourself!** Monetary values are often plotted on logarithmic scales. The `plt.xscale()` and `plt.yscale()` functions can change the scale of the data. Read up on them online and change both to a logarithmic scale!

In [ ]:
# YOUR CODE HERE

❓ **Try it yourself!** There are two other useful arguments for `plt.scatter()`. The `alpha` argument controls the opacity of points, and the `marker` argument controls the shape of the point. Read up on these online, and then set the opacity to 10% and the marker to a cross.

In [ ]:
# YOUR CODE HERE

## Box Plot

A box plot, similar to a histogram, provides the distribution of a continuous variable. The reason we might use box plots is to visualize the distribution of a continuous variable, separated by discrete categories.

In `matplotlib`, the `plt.boxplot()` function creates box plots. The function expects a list of lists. Each list within the grand list contains data for one box plot. For example, the list

```
[[1, 2, 3], [4, 5]]
```

will create two box plots - one with the data `[1, 2, 3]`, and the other with the data `[4, 5]`.

We will plot the distribution of the log-normalized median house value against bins of median age. These bins have already been created when we processed the data.

To create the list of lists, I start by identifying all the bins using some `pandas` code and store it in `median_age_cats`. Print out the value to see what is stored in it!

Then, I use some `pandas` filtering steps to separate the `median_house_value` column based on the bin of `median_age`.

In [ ]:
median_age_cats = cal_housing.median_age.cat.categories.values

plot_data = list()
for median_age in median_age_cats:
  cat_median_house_value = cal_housing.median_house_value[cal_housing.median_age == median_age]
  plot_data.append(np.log(cat_median_house_value.values))

❓ **Try it yourself!** Print out the values of `median_age_cats` and `plot_data` to understand how the code above works.

In [ ]:
# YOUR CODE HERE

Once you have formatted the data properly, plotting is straightforward with the `plt.boxplot()` function.

In [ ]:
plt.figure(figsize=(8, 8))

plt.boxplot(plot_data, labels=median_age_cats)

plt.xlabel('Median Age Bin')
plt.ylabel('log(Median House Value)')

Changing colors is a little challenging with box plots. First, we have to set the `patch_artist` argument in the `plt.boxplot()` function to indicate that we want to edit the graphics further.

The resulting object, which we store in `box_plot`, contains two items stored under the keys of `'boxes'` and `'medians'`. These are the graphic objects for the boxes and median lines respectively. We can loop through these and set the colors as necessary.

In [ ]:
plt.figure(figsize=(8, 8))

box_plot = plt.boxplot(plot_data, patch_artist=True, labels=median_age_cats)

for box in box_plot['boxes']:
  box.set_facecolor('firebrick')
for median in box_plot['medians']:
  median.set_color('black')

plt.xlabel('Median Age Bin')
plt.ylabel('log(Median House Value)')

❓ **Try it yourself!** Choose six colors. Set the color of each box to a unique color, and set the median line color to `'black'`.

In [ ]:
# YOUR CODE HERE

## Heatmaps

Heatmaps are great for visualizing matrices. These are becoming increasingly popular in visualizing large genomics data sets, for example. Here, we will practice using the correlation matrix, which quickly summarizes the strength of lienar relationships between the various features in our data set.

First, I select a subset of columns that I think will be interesting to visualize. The resulting data frame is stored in `variables`.

The `DataFrame` object has a very useful `corr()` function, which estimates the sample correlation matrix. This is stored in the `corr_mtx` variable.

In [ ]:
variables = cal_housing[[
    'housing_median_age', 'total_rooms', 'total_bedrooms', 'population',
    'households', 'median_income', 'median_house_value'
]]

corr_mtx = variables.corr()

corr_mtx

The quickest way to plot this data is using the `plt.imshow()` function.

In [ ]:
plt.figure(figsize=(8, 8))

plt.imshow(corr_mtx)

Unfortunately, this does not tell us which variables each row/column represent. To add these, we use the `plt.xticks()` and `plt.yticks()` functions. For the x-axis, we additionally rotate the labels to help them fit onto the plot.

In [ ]:
variables = corr_mtx.columns.values

plt.figure(figsize=(8, 8))

plt.imshow(corr_mtx)

plt.xticks(range(len(variables)), labels=variables, rotation=45, ha='right', rotation_mode='anchor')
plt.yticks(range(len(variables)), labels=variables)

The heatmap cannot be interpreted without a color bar, which indicates what different colors represent. The `plt.colorbar()` function creates a colorbar for the heatmap.

In [ ]:
variables = corr_mtx.columns.values

plt.figure(figsize=(8, 8))

plt.imshow(corr_mtx)

plt.xticks(range(len(variables)), labels=variables, rotation=45, ha='right', rotation_mode='anchor')
plt.yticks(range(len(variables)), labels=variables)

plt.colorbar()

## A Full Example

In this example, we will create a plot to show the results of linear regression on a subset of the data. The tools required to perform linear regression are available in the `numpy` and `scipy` libraries.

In [ ]:
import scipy as sp
import numpy as np

We will regress log-transformed median house value on log-trasnformed median income. We will use `np.random.choice()` to choose 30 random records from the data set. The `np.random.seed()` function allows us to replicate the chosen records.

In [ ]:
plot_data = cal_housing[cal_housing.median_house_value < 500_000]

x = np.log(plot_data.median_income.values)
y = np.log(plot_data.median_house_value.values)

np.random.seed(42)
idx = np.random.choice(np.arange(len(x)), 30, replace=False)
x = x[idx]
y = y[idx]

The `sp.stats.linregress()` function performs simple linear regression between an independent and dependent variable. The object returned by the function contains the results, including the `intercept`, `slope`, and `rvalue` (coefficient of determination).

We extract the intercept as `alpha`, the slope as `beta`, and the number of samples as `n`.

In [ ]:
lin_res = sp.stats.linregress(x, y)
alpha = lin_res.intercept
beta = lin_res.slope
n = len(x)

We perform some calculations to estimate the confidence band around the linear model under a standard assumption of normality. Briefly, the confidence band is

$$\begin{align}
\alpha + \beta x_0 &\in \left( \hat{\alpha} + \hat{\beta} x_0 \right) \pm t^*_{n-2} \sqrt{ \left( \frac{1}{n-2} \sum_{i=1}^n \hat{\epsilon}_i^2 \right) \cdot \left( \frac{1}{n} + \frac{ \left( x_0 - \bar{x} \right)^2 }{ \sum_{i=1}^n \left( x_i - \bar{x} \right)^2 } \right) }\\
&\in \left( \hat{\alpha} + \hat{\beta} x_0 \right) \pm t^*_{n-2} \sqrt{ \frac{\mathrm{RSS}}{n-2} \left( \frac{1}{n} + \frac{ \left( x_0 - \bar{x} \right)^2 }{ \mathrm{XMSE} } \right) }\\
\end{align}$$

for some test point $x_0$, as discussed [here](https://en.wikipedia.org/wiki/Simple_linear_regression#Normality_assumption).

In [ ]:
# Critical value from the T distribution
t_crit = sp.stats.t.ppf(1 - (0.05 / 2), n - 2)

# Regression Sum of Squares (RSS)
# X Mean Squared Error (XMSE)
rss = np.sum((y - (alpha + beta * x))**2)
xmse = np.sum((x - np.mean(x))**2)

# Create a grid of test points to plot
test_x = np.linspace(x.min(), x.max(), 301)

# Create the interval for the test points
interval = t_crit * np.sqrt(rss / (n - 2)) * ((1 / n) + ((test_x - np.mean(x))**2) / xmse)

The `plt.plot()` function creates a connected curve based on the points provided. We can use this to make a line.

The `plt.fill_between()` function creates a "ribbon" that is filled. This allows us to plot the confidence band around the fitted model.

The `plt.text()` function allows us to place arbitrary text in the figure, which we can use to plot the equation of the fit model and the $R^2$ value.

In [ ]:
line_eq = f'y = {alpha:.2f} + {beta:.2f}x'
r2 = f'R^2 = {lin_res.rvalue**2:.2f}'

plt.figure(figsize=(6, 6))

plt.scatter(x, y, color='black')
plt.plot(test_x, alpha + beta * test_x, color='firebrick')
plt.fill_between(test_x, alpha + beta * test_x - interval, alpha + beta * test_x + interval, alpha=0.2, color='firebrick')
plt.text(1.25, 11.25, line_eq)
plt.text(1.25, 11.15, r2)

plt.xlabel('log(Median Income)')
plt.ylabel('log(Median House Value)')